In [7]:
import math
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm import tqdm

from transformers import BertConfig, BertModel
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

#device = torch.device("mps")

# 데이터 불러오기 (시계열 가격 데이터)
data = pd.read_csv('./processed_data/train_merge.csv')

In [27]:
item_lst

array(['TG', 'CR', 'CB', 'RD', 'BC'], dtype=object)

In [14]:
item_lst = data['item'].unique()

# for item in item_lst:
#     data[data['item']==item].to_csv(f'./processed_data/{item}_train_merge.csv')

In [16]:
# data = pd.read_csv('./data/train.csv')

# # %%
# time = pd.to_datetime(data['timestamp'].copy())

# data['Date'] = pd.to_datetime(data['timestamp'])
# data['week'] = data['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
# data['day_name'] = data['Date'].dt.day_name()

# data['year'] = data['timestamp'].apply(lambda x : int(x[0:4]))
# data['month'] = data['timestamp'].apply(lambda x : int(x[5:7]))
# data['day'] = data['timestamp'].apply(lambda x : int(x[8:10]))
item = item_lst[0]
data = pd.read_csv(f'./processed_data/{item}_train_merge.csv')

X = data.drop(columns=['ID', 'timestamp', 'supply(kg)_x', 'price(원/kg)', '기간', '품목명'])
Y = data['price(원/kg)']

#질적 변수들을 수치화합니다
qual_col = ['item', 'corporation', 'location', 'day_name']

for i in qual_col:
    le = LabelEncoder()
    X[i]=le.fit_transform(X[i])


In [17]:
X = X.drop(columns=['Date'])
X = X


In [18]:
# 입력 데이터에 Min-Max 스케일링 적용
scaler = MinMaxScaler()

X = scaler.fit_transform(X)
#X# = X.reshape(X.shape[0], X.shape[1], 1)

# 입력 텍스트와 레이블 생성
Y = np.array(Y).reshape(Y.shape[0], 1)

In [19]:
X.shape

(15230, 16)

In [20]:
# 데이터를 훈련 세트와 테스트 세트로 분할합니다.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
# # 모델 정의 트랜스포머
# class TimeSeriesTransformer(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, num_heads):
#         super(TimeSeriesTransformer, self).__init__()
        
#         # Transformer 모델 불러오기
#         transformer_config = BertConfig(
#             hidden_size=hidden_size,
#             num_hidden_layers=num_layers,
#             num_attention_heads=num_heads,
#             intermediate_size=hidden_size * 4,
#             hidden_dropout_prob=0.1,
#             attention_probs_dropout_prob=0.1,
#         )
#         self.transformer = BertModel(transformer_config)
        
#         # Fully Connected Layer 추가
#         self.fc1 = nn.Linear(hidden_size, hidden_size//2)
#         self.fc2 = nn.Linear(hidden_size//2, hidden_size//4)
#         self.fc3 = nn.Linear(hidden_size//4, 1)
    
#     def forward(self, x, attention_mask=None):
#         outputs = self.transformer(x, attention_mask=attention_mask)
#         pooled_output = outputs.last_hidden_state.mean(1)  # 각 시퀀스의 평균을 사용
#         out = self.fc1(pooled_output)
#         out = self.fc2(out)
#         out = self.fc3(out)
#         return out


In [22]:
import torch.nn as nn
from torch.nn.modules import Transformer
import torch_geometric.nn as gnn

# 모델 정의 LSTM with Multi-Head Self-Attention
class TimeSeriesLSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, nhead):
        super(TimeSeriesLSTMWithAttention, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size*4, num_layers, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size*4, hidden_size*2, num_layers, batch_first=True)
        
        self.fc1 = nn.Linear(hidden_size*4, hidden_size*2)
        self.fc2 = nn.Linear(hidden_size*2, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        # out, _ = self.lstm3(out)
        #out, _ = self.lstm2(out)
        
        # Multi-Head Self-Attention 적용

        #out = self.fc1(out[:, -1, :])
        out = self.fc2(out[:, -1, :])
        out = self.fc3(out)
        return out


In [23]:
import torch.nn as nn
from torch.nn.modules import Transformer
import torch_geometric.nn as gnn

# 모델 정의 LSTM with Multi-Head Self-Attention
class TimeSeriesDNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, nhead):
        super(TimeSeriesDNN, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_size*16)

        self.fc2 = nn.Linear(hidden_size*16, hidden_size*16)
        self.fc3 = nn.Linear(hidden_size*16, hidden_size*8)

        self.fc4 = nn.Linear(hidden_size*8, hidden_size*8)
        self.fc5 = nn.Linear(hidden_size*8, hidden_size*4)

        self.fc6 = nn.Linear(hidden_size*4, hidden_size*4)
        self.fc7 = nn.Linear(hidden_size*4, hidden_size*2)
  
        self.fc8 = nn.Linear(hidden_size*2, hidden_size*2)
        self.fc9 = nn.Linear(hidden_size*2, hidden_size)

        self.fc10 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        out = self.fc5(out)
        out = self.fc6(out)
        out = self.fc7(out)
        out = self.fc8(out)
        out = self.fc9(out)
        out = self.fc10(out)
        return out


In [24]:
# import torch.nn as nn
# from torch.nn.modules import Transformer

# # 모델 정의 Transformer with Multi-Head Self-Attention and Conv1d
# # 모델 정의 Transformer with Multi-Head Self-Attention and Conv1d
# class TimeSeriesTransformerWithAttentionAndConv1d(nn.Module):
#     def __init__(self, d_model, nhead, num_filters, kernel_size):
#         super(TimeSeriesTransformerWithAttentionAndConv1d, self).__init__()
#         self.transformer = Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=3)  # Transformer 레이어 추가
#         self.conv1d = nn.Conv1d(in_channels=d_model, out_channels=num_filters, kernel_size=kernel_size)

#         self.fc1 = nn.Linear(num_filters, d_model)
#         self.fc2 = nn.Linear(d_model, d_model // 2)
#         self.fc3 = nn.Linear(d_model // 2, 1)

#     def forward(self, x):
#         # 'src'와 'tgt'의 feature 차원을 동일하게 맞춤
#         src = x
#         tgt = x
#         out = self.transformer(src, tgt)  # Transformer 레이어 적용

#         # Conv1d 레이어를 통해 공간적 패턴을 감지
#         out = out.permute(0, 2, 1)  # 컨볼루션을 위해 차원 변경
#         out = self.conv1d(out)

#         out = out.permute(0, 2, 1)  # 다시 차원 변경
#         out = self.fc1(out)
#         out = self.fc2(out)
#         out = self.fc3(out)
#         return out


In [25]:
import torch
import torch.nn as nn

class TimeSeriesCNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(TimeSeriesCNN, self).__init__()

        # 1D CNN 레이어 정의
        self.cnn1 = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=hidden_size*2, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size*2)
        )

        self.cnn2 = nn.Sequential(
            nn.Conv1d(in_channels=hidden_size*2, out_channels=hidden_size, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size)
        )

        self.cnn3 = nn.Sequential(
            nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size)
        )

        # Fully Connected Layer 추가
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.fc3 = nn.Linear(hidden_size // 4, 1)

    def forward(self, x):
        # 입력 텐서 차원을 (배치 크기, 시간 단계, 피쳐)로 변경
        x = x.permute(0, 2, 1)

        out = self.cnn1(x)
        out = self.cnn2(out)
        out = self.cnn3(out)

        # CNN 이후 다시 차원을 변경하여 Fully Connected Layer에 적용
        out = out.permute(0, 2, 1)

        out = self.fc1(out[:, -1, :])
        out = self.fc2(out)
        out = self.fc3(out)
        return out


In [26]:
# LSTM

input_size = X_train.shape[1]
hidden_size = 32
num_layers = 2
#model = TimeSeriesLSTMWithAttention(input_size, hidden_size, num_layers, 2)
model = TimeSeriesDNN(input_size, hidden_size, num_layers, 2)
#model = TimeSeriesCNN(input_size, hidden_size, num_layers)

# # 모델 파라미터 정의
# input_size = X_train.shape[2]  # 입력 데이터의 특성 수
# d_model = 64  # Transformer 레이어의 출력 차원
# nhead = 4  # Multi-Head Self-Attention의 헤드 수
# num_filters = 32  # Conv1d의 출력 채널 수
# kernel_size = 3  # Conv1d의 커널 크기
# model = TimeSeriesTransformerWithAttentionAndConv1d(d_model=64, nhead=4, num_filters=32, kernel_size=3)

# 모델 훈련 설정
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

#criterion = torch.nn.MSELoss()
criterion = torch.nn.MSELoss()

# Validation 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Early stopping 관련 설정
best_val_loss = float('inf')
patience = 30  # 일정 횟수 동안 검증 손실이 향상되지 않을 때 조기 종료
counter = 0
epoch = 0

while 1:
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Validation 손실 확인
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
    epoch += 1
    print(f"epoch : {epoch} / Train loss : {math.sqrt(criterion(outputs, y_train_tensor))} / Validation loss: {math.sqrt(criterion(val_outputs, y_val_tensor))}")
        # 검증 손실이 이전 최고 손실보다 낮으면 모델 가중치 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), './weights/best_model.pth')
        counter = 0
    else:
        counter += 1
    
    # 검증 손실이 일정 횟수 동안 향상되지 않으면 조기 종료
    if counter >= patience:
        print("Early stopping.")
        break


epoch : 1 / Train loss : 4239.566723145184 / Validation loss: 4273.771636388636
epoch : 2 / Train loss : 4239.502093406724 / Validation loss: 4273.645750410298
epoch : 3 / Train loss : 4239.375897464154 / Validation loss: 4273.340847627299
epoch : 4 / Train loss : 4239.070416966437 / Validation loss: 4272.629401200155
epoch : 5 / Train loss : 4238.35770080818 / Validation loss: 4271.081596036302
epoch : 6 / Train loss : 4236.807524540146 / Validation loss: 4267.915416219023
epoch : 7 / Train loss : 4233.636262127393 / Validation loss: 4261.755976120641
epoch : 8 / Train loss : 4227.467563447412 / Validation loss: 4250.26375652147
epoch : 9 / Train loss : 4215.9584912567625 / Validation loss: 4229.53117969356
epoch : 10 / Train loss : 4195.198922578046 / Validation loss: 4193.26507628602
epoch : 11 / Train loss : 4158.8907174870565 / Validation loss: 4131.625346035141
epoch : 12 / Train loss : 4097.194650001388 / Validation loss: 4029.8872192655713
epoch : 13 / Train loss : 3995.3994794

In [119]:
# 테스트 데이터로 평가
model.eval()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    mse = mean_squared_error(y_test_tensor, test_outputs.numpy())

print("평균 제곱 오차 (RMSE):", math.sqrt(mse))

평균 제곱 오차 (RMSE): 1729.6001705596586


In [ ]:

# # Transformer
# # 모델 생성 및 훈련
# input_size = X_train.shape[1]
# hidden_size = 64
# num_layers = 2
# num_heads = 8
# model = TimeSeriesTransformer(input_size, hidden_size, num_layers, num_heads)

# # 모델 훈련 설정
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
# criterion = torch.nn.MSELoss()

# # Validation 데이터 분할
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
# y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# # Early stopping 관련 설정
# best_val_loss = float('inf')
# patience = 10  # 일정 횟수 동안 검증 손실이 향상되지 않을 때 조기 종료
# counter = 0

# for epoch in range(1000):
#     model.train()
#     optimizer.zero_grad()
#     outputs = model(X_train_tensor)
#     loss = criterion(outputs, y_train_tensor)
#     loss.backward()
#     optimizer.step()

#     # Validation 손실 확인
#     model.eval()
#     with torch.no_grad():
#         val_outputs = model(X_val_tensor)
#         val_loss = criterion(val_outputs, y_val_tensor)
    
#     print(f"epoch : {epoch} / Train loss : {math.sqrt(loss)} / Validation loss: {math.sqrt(val_loss)}")

#         # 검증 손실이 이전 최고 손실보다 낮으면 모델 가중치 저장
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         torch.save(model.state_dict(), './weights/best_model.pth')
#         counter = 0
#     else:
#         counter += 1
    
#     # 검증 손실이 일정 횟수 동안 향상되지 않으면 조기 종료
#     if counter >= patience:
#         print("Early stopping.")
#         break
